# Handling Mislabeled Tabular Data to Improve Your XGBoost Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cleanlab/examples/blob/master/find_tabular_errors/find_tabular_errors.ipynb)

This article highlights data-centric AI techniques using [cleanlab](https://github.com/cleanlab/cleanlab) to improve the accuracy of an XGBoost classifier (reducing prediction errors by 70% on the noisy dataset considered here!). These techniques optimize the dataset itself rather than altering the model’s architecture or hyperparameters. As a result, it is possible to achieve further improvements in accuracy by fine-tuning the model with the newly enhanced data. Enhancements to the dataset are model-agnostic and therefore are transferable to other modeling and analytical endeavors, as opposed to being specific to a particular type of model.

At a high level we will:
- Establish a baseline XGBoost model accuracy trained on the original data.
- Use cleanlab's `find_label_issues()` to highlight hundreds of mislabeled data points. 
- Remove the data with automatically-flagged label issues from the dataset, and then retrain the exact same XGBoost model. **This simple step reduces the error in model predictions by 36%!** The raw difference in accuracy values between the two XGBoost models is **8%**.
- Introduce a **no-code** solution to efficiently fix the label errors in the dataset which **reduces the error in model predictions by 70%** from the baseline, identical XGBoost model!

## Setup and Data Processing

Let’s take a look at our student grades tabular dataset. The data includes three exam scores (numerical features), a written note (categorical feature with missing values), and a (noisy) letter grade (categorical label). Our aim is to train a model to classify the grade for each student based on the other features, but 20% of the grade labels in this dataset are actually incorrect.

We have access to the true letter grade each student should’ve received, which we use for evaluating both the underlying accuracy of model predictions and how well cleanlab detects which data are mislabeled. These true grades are only reserved for model evaluation and are manually validated, gold-standard labels. They are not present in any of the training procedures. We utilize a 75/25 split for out train/test data.

In your noisily-labeled datasets, there will typically be no such ground truth, and therefore addressing label issues is even more important to facilitate proper model evaluation.

Please install the dependencies specified in this [requirements.txt](https://github.com/cleanlab/examples/blob/master/find_tabular_errors/requirements.txt) file before running the notebook.

In [ ]:
from cleanlab.filter import find_label_issues
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("https://s.cleanlab.ai/student-grades-demo.csv")
df_c = df.copy()

# Transform letter grades and notes to categorical numbers.
# Necessary for XGBoost and cleanlab.
df['letter_grade'] = preprocessing.LabelEncoder().fit_transform(df['letter_grade'])
df['noisy_letter_grade'] = preprocessing.LabelEncoder().fit_transform(df['noisy_letter_grade'])
df['notes'] = preprocessing.LabelEncoder().fit_transform(df["notes"])
df['notes'] = df['notes'].astype('category')

# Split data for evaluation and set test data.
df_train, df_test = train_test_split(df, random_state=0)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
test_data = df_test.drop(['stud_ID', 'letter_grade', 'noisy_letter_grade'], axis=1)
test_labels = df_test['letter_grade']

# Train and Evaluate XGBoost Classifier

Now that we’ve seen what can be achieved with cleanlab, let’s take a look at how we get there.

For our model of choice, we will use XGBoost, an implementation of gradient-boosting decision trees (GBDT), which are commonly used with tabular data. If our tabular data consisted solely of numerical and boolean values, we could potentially utilize a simpler model such as a nearest-neighbor or logistic regression. However, our data includes a notes column, which we will treat as a categorical feature. Fortunately, XGBoost (>v1.6) is able to handle mixed data types (numerical and categorical) by setting the `enable_categorical` parameter to `true`, thereby simplifying the modeling process.

In [ ]:
# Train model on noisy labels.
train_data = df_train.drop(['stud_ID', 'letter_grade', 'noisy_letter_grade'], axis=1)
train_labels = df_train['noisy_letter_grade']

# XGBoost(experimental) supports categorical data.
# Here we use default hyperparameters for simplicity.
model = XGBClassifier(tree_method="hist", enable_categorical=True)
model.fit(train_data, train_labels)

# Evaluate model on test split with ground truth labels.
preds = model.predict(test_data)
acc_original = accuracy_score(preds, test_labels)
print(f"Accuracy with original data: {round(acc_original*100,1)}%")

Accuracy with original data: 79.2%


Using the default hyperparameters, our baseline XGBoost model demonstrates an accuracy of 79.2% when trained on the noisy labels and predicting the test set. It appears that the presence of 20% label noise is significantly disrupting the model’s ability to accurately predict the labels on such a trivial task.

# Find Label Issues

In order to use cleanlab, we need to obtain **out-of-sample** predicted probabilities for all of our training data in order to provide the `find_label_issues()` method with the necessary input. Getting the predicted probabilities can be achieved through the use of our `XGBClassifier` model with cross-validation, which can be implemented easily using the `cross_val_predict` function from scikit-learn.

In just a few lines of code, we get a list of possible label issues! A few of the top results are shown below.

In [ ]:
# Get predicted probabilities through cross validation.
model = XGBClassifier(tree_method="hist", enable_categorical=True)
pred_probs = cross_val_predict(model, train_data, train_labels, method='predict_proba')

# Returns list of indices of label issues, sorted by self_confidence.
issue_idx = find_label_issues(train_labels, pred_probs, return_indices_ranked_by='self_confidence')

# Filter original data to show students with grade issues.
issue_stud_id = df_train.iloc[issue_idx].stud_ID.values.tolist()
issues_df = df_c[df_c['stud_ID'].isin(issue_stud_id)]
issues_df.sort_values(by="stud_ID", key=lambda column: column.map(lambda e: issue_stud_id.index(e)), inplace=True)

# Show a few good examples.
issues_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,stud_ID,exam_1,exam_2,exam_3,notes,letter_grade,noisy_letter_grade
765,75ce98,91,89,81,NaN,B,F
744,3d0fdf,90,74,95,great participation +10,A,F
637,77c9c5,0,79,65,"cheated on exam, gets 0pts",F,A
404,bb13f4,65,95,68,NaN,C,A
217,3c4cbb,94,62,66,NaN,C,C


Let’s take a look at a few of the label issues automatically identified in our dataset. Take a look at row 1, where the student got grades of 91, 89, and 81, which should result in a ‘B’ yet was accidentally labeled as an ‘F’. In row 2, the student had great participation resulting in an addition of 10 points to the overall average, receiving exam grades of 90, 74, and 95 (averages to 86.3, overall 96.3 with the bonus), which should result in a ‘A’ yet was accidentally labeled as an ‘F’.

**Note: `find_label_issues` is able to determine that the given label is incorrect, without ever seeing the ground truth label `letter_grade`.**

# How'd We Do?

Let's go a step further and see how cleanlab did at automatically identifying which data points are mislabeled. If we take the intersection of the label errors identified by cleanlab and the true label errors, we see that cleanlab was able to identify 83% of the label errors correctly (based on predictions from a model that is only 79% accurate). 

In [ ]:
# Computing percentage of true errors identified. 
true_error_idx = df_train[df_train.letter_grade != df_train.noisy_letter_grade].index.values
cl_acc = len(set(true_error_idx).intersection(set(issue_idx)))/len(true_error_idx)
print(f"Percentage of errors found: {round(cl_acc*100,1)}%")

Percentage of errors found: 82.9%


# Retraining for a More Robust Model

Now that we have the indices of potential label errors let’s remove them from our data, retrain our model, and see what performance improvement we can gain.

Keep in mind our baseline model from above, trained on the original data using the `noisy_letter_grade` as the prediction label, achieved an accuracy of 79%.

Let’s use a very simple method to handle these label errors and just drop them entirely from the data and retrain our exact same `XGBClassifier`.

In [ ]:
# Remove the label errors found by cleanlab.
train_data_cl = df_train.drop(issue_idx)
train_labels_cl = train_data_cl['noisy_letter_grade']
train_data_cl = train_data_cl.drop(['stud_ID', 'letter_grade', 'noisy_letter_grade'], axis=1)

# Train a more robust classifier with less erroneous data.
model = XGBClassifier(tree_method="hist", enable_categorical=True)
model.fit(train_data_cl, train_labels_cl)

# Evaluate model on test split with ground truth labels.
preds = model.predict(test_data)
acc_clean = accuracy_score(preds, test_labels)
print(f"Accuracy with original data: {round(acc_original*100, 1)}%")
print(f"Accuracy with errors found by cleanlab removed: {round(acc_clean*100, 1)}%")

# Compute reduction in error.
err = ((1-acc_original)-(1-acc_clean))/(1-acc_original)
print(f"Reduction in error: {round(err*100,1)}%")

Accuracy with original data: 79.2%
Accuracy with errors found by cleanlab removed: 86.9%
Reduction in error: 36.7%


After removing the suspected label issues, our model's new accuracy is now 87%, which means we **reduced the error-rate of the model by 36%**. 

**Note: throughout this entire process, we never changed any code related to model architecture/hyperparameters, training, or data preprocessing! This improvement is strictly coming from increasing the quality of our data which leaves room for additional optimizations on the modeling side.**

# Fixing the Label Errors

Instead of just dropping the potential label issues, the smarter (yet more complex) way to increase our data quality would be to correct the label issues by hand. This simultaneously removes a noisy data point and adds an accurate one, but making such corrections manually is cumbersome.

[Cleanlab Studio](https://cleanlab.ai/studio) provides a user-friendly interface to make these changes without writing a single line of code. Simply upload your dataset and Studio computes everything we just did for you, so you can spend more time fixing the issues instead of just finding them.

Here, we use the auto-fix feature on this dataset and replace the Studio-found label issues with the automatically-suggested label. From data upload to data export, the whole process took only 5 minutes without having to know any ML.  


In [ ]:
# Get the export produced by Cleanlab Studio and split to match training subset.
clean_df = pd.read_csv("https://s.cleanlab.ai/student-grades-demo-studio-export.csv")
train_students = df_train.stud_ID.values
clean_df = clean_df[clean_df['stud_ID'].isin(train_students)]

# Same pre-processing as above.
clean_df['cleanlab_suggested_label'] = preprocessing.LabelEncoder().fit_transform(clean_df['cleanlab_suggested_label'])
clean_df['notes'] = preprocessing.LabelEncoder().fit_transform(clean_df["notes"])
clean_df['notes'] = clean_df['notes'].astype('category')

# Train a more robust classifier with less erroneous data.
clean_labels = clean_df['cleanlab_suggested_label']
clean_data = clean_df[['exam_1','exam_2','exam_3','notes']]
model = XGBClassifier(tree_method="hist", enable_categorical=True)
model.fit(clean_data, clean_labels)

# Evaluate model on test split with ground truth labels.
preds = model.predict(test_data)
acc_studio = accuracy_score(preds, test_labels)
print(f"Accuracy with original data: {round(acc_original*100, 1)}%")
print(f"Accuracy with errors found by cleanlab removed: {round(acc_clean*100, 1)}%")
print(f"Accuracy with errors found by Studio fixed: {round(acc_studio*100, 1)}%")
print()

# Compute reductions in error.
clos_err = ((1-acc_original)-(1-acc_clean))/(1-acc_original)
studio_err = ((1-acc_clean)-(1-acc_studio))/(1-acc_clean)
tot_err = ((1-acc_original)-(1-acc_studio))/(1-acc_original)
print(f"Reduction in error using cleanlab opensource over baseline: {round(clos_err*100,1)}%")
print(f"Reduction in error using cleanlab Studio over opensource: {round(studio_err*100,1)}%")
print(f"Reduction in error using Cleanlab Studio over baseline: {round(tot_err*100,1)}%")

Accuracy with original data: 79.2%
Accuracy with errors found by cleanlab removed: 86.9%
Accuracy with errors found by Studio fixed: 93.6%

Reduction in error using cleanlab opensource over baseline: 36.7%
Reduction in error using cleanlab Studio over opensource: 51.6%
Reduction in error using Cleanlab Studio over baseline: 69.4%


# Conclusion

Cleanlab is an incredibly powerful and efficient tool for identifying and addressing label errors in your data that can be used to improve any ML model (not just XGBoost) for most types of data (not just tabular, but also images, text, audio, etc). By implementing just a few lines of open-source code, cleanlab can automatically detect and help you prioritize many potential issues within your data. With this insight, you'll be able to improve the quality of your data and ultimately achieve better model performance.

For the student grades dataset, we found that **simply dropping identified label errors and retraining the model resulted in a 36% reduction in prediction error** on our classification problem (with accuracy improving from 79% to 87%). 

Going one step further, we used Cleanlab Studio to automatically **fix the incorrect labels,resulting in a 70% reduction in prediction error** (with accuracy improving from 79% to 94%).

By using open-source libraries for data-centric AI like [cleanlab](https://github.com/cleanlab/cleanlab) to ensure the integrity of your data, you can mitigate costly labeling errors and boost the performance of your models.


# Next Steps

We would like to extend a special thanks to all of our open-source contributors. Your support and engagement have played a crucial role in the development and success of cleanlab. If you are interested in becoming a contributor to the cleanlab project and helping us build the standard open-source library for data-centric AI, please visit our [GitHub page](https://github.com/cleanlab/cleanlab) and [contributing guide](https://github.com/cleanlab/cleanlab/blob/master/CONTRIBUTING.md).

If you interested in using cleanlab to improve your data-centric techniques and ML tasks, our comprehensive [tutorials](https://docs.cleanlab.ai/) provide a simple and efficient way to get started. In just 5 minutes, you can learn how to apply cleanlab to a variety of data types (text, tabular, image, audio, etc) and ML tasks (classification, entity recognition, image/document tagging, etc).

We would love to connect with you, too!
- Join our [Cleanlab Community Slack](https://cleanlab.ai/slack/)
- Follow us on [LinkedIn](https://www.linkedin.com/company/cleanlab/)
- Follow us on [Twitter](https://twitter.com/CleanlabAI)

Bonus: Learn how cleanlab can also help improve training data in [Kaggle](https://www.kaggle.com/code/ulytkch/cleanlab-data-centric-ai-example-0-7703-python/notebook) competitions.
